<a href="https://colab.research.google.com/github/alfiindraini/Analisis-sentimen/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **Library**

In [ ]:
pip install Pysastrawi

In [ ]:
import pandas as pd
import numpy as np
import re
import textwrap
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_confusion_matrix
from sklearn.preprocessing import normalize
from sklearn.svm import SVC

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# **Baca Data**

In [ ]:
data_awal = pd.read_excel('/content/drive/My Drive/skripsi/data/data_awal.xlsx')

In [ ]:
df = pd.DataFrame(data_awal) 

In [ ]:
df['Label'] = df['Label'].replace('negatif',0)
df['Label'] = df['Label'].replace('positif',1)

# **Pre-Processing**

In [ ]:
daftar_emot = {
    '😀' : ' senang ',
    '😃' : ' senang ',
    '😄' : ' senang ',
    '😁' : ' senang ',
    '😆' : ' senang ',
    '😅' : ' senang ',
    '😂' : ' senang ',
    '🤣' : ' senang ',
    '😉' : ' senang ',
    '😊' : ' senang ',
    '🙂' : ' senang ',
    '😍' : ' senang ',
    '🙁' : ' kecewa ',
    '🥺' : ' mohon ',
    '😭' : ' sedih ',
    '😥' : ' kecewa ',
    '😫' : ' lelah ',
    '😖' : ' kesal ',
    '😣' : ' kesal ',
    '😩' : ' kesal ',
    '🤬' : ' kesal ',
    '😡' : ' kesal ',
    '😠' : ' kesal ',
    '😞' : ' kecewa ',
    '😤' : ' kesal ',
    '😒' : ' kesal ',
    '😔' : ' kecewa ',
    '❤️' : ' senang ',
    '💔' : ' kecewa',
    
}


In [ ]:
kamus_kata = {
#' an ' : ' ',
' adu ' : ' lawan ',
' bobok ' : ' tidur ',
' buk ' : ' ibu ',
' memang ' : ' sebenarnya ',
' sih ' : ' sebenarnya ',
' emang ' : ' benar ',
' abes ' : ' habis ',
' abis ' : ' habis ',
' abrek ' : ' hancur ',
' abs ' : ' abis ',
' absen ' : ' hadir ',
' acu ' : ' aku ',
' adek ' : ' adik ',
' adh ' : ' waduh ',
'bahagia' : 'senang',
' adminya ' : ' admin ',
' ads ' : ' ada ',
' again ' : ' lagi ',
' agak ' : ' sedikit ',
' ah ' : ' kecewa ',
' ahlak ' : ' perilaku ',
' aing ' : ' saya ',
' aj ' : ' saja ',
' aja ' : ' saja ',
' ake ' : ' pake ',
' akhir ' : ' akhir ',
' aks ' : ' alias ',
' aku ' : ' saya ',
' akutuh ' : ' saya ',
' akuw ' : ' saya ',
' alami ' : ' alam ',
' alesan ' : ' alasan ',
' almost ' : ' hampir ',
' alpa ' : ' bolos ',
' cabut ' : ' bolos ',
' also ' : ' juga ',
' ama ' : ' sama ',
' amp ' : ' sampai ',
' ampe ' : ' sampai ',
' anakya ' : ' anak ',
' and ' : ' dan ',
' angota ' : ' anggota ',
' anh ' : ' aneh ',
' anj ' : ' anjing ',
' anjir ' : ' kesal ',
' anjrit ' : ' kesal ',
' anjlok ' : ' turun ',
' ank ' : ' anak ',
' anoying ' : ' kesal ',
' anteng ' : ' damai ',
' anter ' : ' antar ',
' anything ' : ' segala sesuatu ',
' ap ' : ' apa ',
' apdet ' : ' terkini ',
' ape ' : ' apa ',
' as ' : ' sebagai ',
' asis' : ' asik ',
' ato ' : ' atau ',
' atsan ' : ' atasan ',
' auah ' : ' tidak tahu ',
' auto ' : ' bekerja sendiri ',
' back ' : ' kembali ',
' bakal ' : ' dibuat ',
' bales ' : ' balas ',
' balik ' : ' kembali ',
' bandel ' : ' nakal ',
' kane ' : ' enak ',
' banget ' : ' sangat ',
' bangke ' : ' bangsat ',
' bangrt ' : ' sangat ',
' bangt ' : ' sangat ',
' bapa ' : ' bapak ',
' bareng ' : ' bersama ',
' barusn ' : ' barusan ',
' base ' : ' dasar ',
' batre ' : ' daya ',
' bayik ' : ' bayi ',
' bcs ' : ' karena ',
' bdr ' : ' belajar dari rumah ',
' be ' : ' menjadi ',
' sabeb ' : ' bebas ',
' bego ' : ' bodoh ',
' fak ' : ' bangsat ',
' wtf ' : ' bodoh ',
' beh ' : ' waduh ',
' belakanh ' : ' belakang ',
' belom ' : ' belum ',
' belsajar ' : ' belajar ',
' ben ' : ' benar ',
'bener' : 'benar',
' bentrok ' : ' tabrak ',
' berfikir ' : ' berpikir ',
' bersodara ' : ' bersaudara ',
' bertanguh ' : ' bertanggung ',
' bertangung ' : ' bertanggung ',
' bestie ' : ' sahabat ',
' bestnya ' : ' terbagus ',
' bete ' : ' bosan ',
' beuh ' : ' waduh ',
' bgnya ' : ' latarnya ',
' bgst ' : ' bangsat ',
' bgt ' : ' sangat ',
' bgun ' : ' bangun ',
' bgus ' : ' bagus ',
' bhay ' : ' pergi ',
' bhkn ' : ' bahkan ',
' bhs ' : ' bahasa ',
' bi ' : ' bahasa indonesia ',
' biar ' : ' supaya ',
' biasain ' : ' biasa ',
' bikin ' : ' buat ',
' bimbel ' : ' kursus ',
' bkin ' : ' buat ',
' blegug ' : ' bodoh ',
' blm ' : ' belum ',
' blom ' : ' belum ',
' blum ' : ' belum ',
' bneran ' : ' beneran ',
' bnget ' : ' sangat ',
' bngst ' : ' bangsat ',
' bnyk ' : ' banyak ',
'bobo ' : ' tidur ',
' bongsor ' : ' tinggi besar ',
' boring ' : ' bosan ',
'bosen' : ' bosan ',
' br ' : ' baru ',
'brainles' : ' bodoh ',
' brand ' : ' merek ',
' brgkat ' : ' berangkat ',
' brmakna ' : ' bermakna ',
#' bro ' : ' ',
' sound ' : ' suara ',
' ngertinya ' : ' paham ',
' brp ' : ' berapa ',
' bs ' : ' bisa ',
' bsk ' : ' besok ',
' bth ' : ' butuh ',
' bu ' : ' ibu ',
' buk ' : ' ibu ',
' budak ' : ' anak ',
' bulying ' : ' mencela ',
'sulit' : 'susah',
' bun ' : ' ibu ',
' but ' : ' tetapi ',
' butut ' : ' jelek ',
' bwsok ' : ' besok ',
' by ' : ' dari ',
' bye ' : ' pergi ',
' ape ' : ' apa ',
' cabar ' : ' sabar ',
' cakap ' : ' cantik ',
' calendar ' : ' kalender ',
' can ' : ' bisa ',
' cangung ' : ' canggung ',
'cape' : ' lelah ',
' atend ' : ' hadir ',
' atention ' : ' perhatian ',
'capek' : ' lelah ',
' caper ' : ' cari perhatian ',
' care ' : ' peduli ',
' catetan ' : ' catatan ',
' cemangat ' : ' semangat ',
' ceo ' : ' pemilik ',
' cepet ' : ' cepat ',
' cewek ' : ' perempuan ',
' chat ' : ' pesan ',
' chatan ' : ' pesan ',
' check ' : ' cek ',
' ciduk ' : ' tertangkap ',
' clas ' : ' kelas ',
' class' : ' kelas ',
' cm ' : ' hanya ',
' cma ' : ' hanya ',
' cok ' : ' bodoh ',
' concern ' : ' perhatian ',
' coronces ' : ' corona ',
' course ' : ' kursus ',
' cowok ' : ' lakilaki ',
' cpt ' : ' cepat ',
' crush ' : ' menghancurkan ',
' culture ' : ' budaya ',
' cuman ' : ' hanya ',
' curhat ' : ' cerita ',
#' cuy ' : ' ',
' cwe ' : ' perempuan ',
' dah ' : ' sudah ',
' dapet ' : ' dapat ',
' dar ' : ' dari ',
'darin ' : ' daring ',
'daringnya ' : ' daring ',
' darting ' : ' marah ',
' deactive ' : ' tidak aktif ',
' dek ' : ' adik ',
' deket ' : ' dekat ',
' deliv ' : ' diterima ',
' depa ' : ' depan ',
' dg ' : ' dengan ',
' dgn ' : ' dengan ',
' dh ' : ' sudah ',
' diajarin ' : ' ajar ',
' dicatet ' : ' catat ',
' die ' : ' dia ',
' dikalahin ' : ' kalah ',
' dikumupulin ' : ' kumpul ',
' dingetin ' : ' mengingatkan ',
' dipisui ' : ' disalahgunakan ',
' diskon ' : ' promo ',
' disruh ' : ' suruh ',
' ditingal ' : ' ditinggal ',
' dkt ' : ' dekat ',
' dl ' : ' dulu ',
' dlm ' : ' dalam ',
' dm ' : ' pesan ',
' dmsukin ' : ' dimasukan ',
' dn ' : ' dan ',
' dngn ' : ' dengan ',
' dom ' : ' domisili ',
' don ' : ' selesai ',
' dongoku ' : ' bodoh ',
' donh ' : ' dong ',
' dont ' : ' jangan ',
' dose ' : ' dosa ',
' down ' : ' jatuh ',
' dpn ' : ' depan ',
'dpt' : 'dapat',
' dr ' : ' dari ',
' drakor ' : ' drama korea ',
' drakoran ' : ' drama korea ',
' drop ' : ' sakit ',
' drpd ' : ' dari pada ',
' drtd ' : ' dari tadi ',
' dsn ' : ' disana ',
' dtg ' : ' datang ',
' duh ' : ' waduh ',
' duid ' : ' uang ',
' efek ' : ' faktor ',
' ejan ' : ' ejaan ',
' ekspetasi ' : ' harapan ',
' elo ' : ' kamu ',
' emg ' : ' sebenarnya ',
' emng ' : ' sebenarnya ',
' encok ' : ' sakit pinggang ',
' ending ' : ' akhir ',
' enga ' : ' tidak ',
' engak ' : ' tidak ',
' ngak ' : ' tidak ',
' eror ' : ' rusak ',
' even ' : ' kegiatan ',
' event ' : ' kegiatan ',
' exam ' : ' uji ',
' excited ' : ' heboh ',
' face ' : ' wajah ',
' faedah ' : ' manfaat ',
' family ' : ' keluarga ',
' fase ' : ' masa ',
' fikir ' : ' pikir ',
' fiks ' : ' menetapkan ',
' fine ' : ' bagus ',
' fix ' : ' menetapkan ',
' for ' : ' untuk ',
' fre ' : ' bebas ',
' fuh ' : ' lelah ',
' ful ' : ' penuh ',
' g ' : ' tidak ',
' ga ' : ' tidak ',
' gaada ' : ' tidak ada ',
' gaberkualitas ' : ' tidak berkualitas ',
' gabisa ' : ' tidak bisa ',
' gabole ' : ' tidak boleh ',
' gabut ' : ' bosan ',
' jenuh ' : ' bosan ',
#' gais ' : ' ',
#' gaiz ' : ' ',
#' guys ' : ' ',
' gak ' : ' tidak ',
' gakan ' : ' tidak akan ',
' gamau ' : ' tidak ingin ',
' game ' : ' permainan ',
' ganteng ' : ' ganteng ',
' gapapa ' : ' tidak apa apa ',
' gapernah ' : ' tidak pernah ',
' gapulang ' : ' tidak pulang ',
' gasemua ' : ' tidak semua ',
' gatau ' : ' tidak tahu ',
' gatel ' : ' gatal ',
' gbs ' : ' tidak bisa ',
' si ' : ' sebenarnya ',
' gegara ' : ' garagara ',
' gelaseh ' : ' gila ',
' gemar ' : ' senang ',
' gemez ' : ' gemas ',
' gereget ' : ' gemas ',
' gile ' : ' gila ',
'gimana' : ' bagaimana ',
' gimans ' : ' bagaimana ',
' gin' : ' begini ',
' gitu ' : ' begitu ',
' gitulo ' : ' begitu ',
' gk ' : ' tidak ',
' gluduk ' : ' petir ',
' gmn ' : ' bagaimana ',
' gni ' : ' begini ',
' go ' : ' pergi ',
' goblog ' : ' bodoh ',
'goblok' : ' bodoh ',
' gogle ' : ' google ',
' gonta ' : ' ganti ',
' gpaham ' : ' tidak paham ',
' gr ' : ' percaya diri ',
' grgr ' : ' sebab ',
' group ' : ' kelompok ',
' grub ' : ' kelompok ',
' grup ' : ' kelompok ',
' gsuka ' : ' tidak suka ',
'gua' : 'saya',
'gue' : 'saya',
' gur ' : ' saya ',
' gurune ' : ' guru ',
' gw ' : ' saya ',
' gwe ' : ' saya ',
' hadeh ' : ' waduh ',
' hadeuh ' : ' waduh ',
' haduh ' : ' waduh ',
#' haha ' : ' ',
#' hai ' : ' ',
#' haih ' : ' ',
#' halo ' : ' ',
' hape ' : ' telepon ',
' hapy ' : ' senang ',
' harianbiasanya ' : ' harian biasanya ',
' harini ' : ' hari ini ',
' hawa ' : ' keadaan ',
#' heh' : ' ',
'hehe' : ' ',
#' hehe' : ' ',
#' hehehe' : ' ',
#' heheu ' : ' ',
#' heheu' : ' ',
#' hei ' : ' ',
' here ' : ' disini ',
' hft ' : ' lelah ',
#' hi ' : ' ',
#' hihi ' : ' ',
' hiyak ' : ' iya ',
#' hm ' : ' ',
' hole ' : ' bersorak ',
' hot ' : ' panas ',
' how ' : ' bagaimana ',
' hp ' : ' telpon ',
' hri ' : ' hari ',
' hrs ' : ' harus ',
#' hshshsh' : ' ',
' ht ' : ' hati ',
' hua' : ' nangis ',
' hue ' : ' saya ',
' hum ' : ' rumah ',
' idea ' : ' ide ',
' iki ' : ' itu ',
' ikt ' : ' ikut ',
' ilang ' : ' hilang ',
' ilkom ' : ' ilmu komunikasi ',
' improvement ' : ' memperbaiki ',
' in ' : ' ini ',
' individu ' : ' pribadi ',
' inet ' : ' internet ',
' info ' : ' informasi ',
'inget' : 'ingat',
' interact ' : ' interaksi ',
' interaktif ' : ' berbicara ',
' it ' : ' itu ',
' iye ' : ' iya ',
' jadual ' : ' jadwal ',
' jajn ' : ' jajan ',
' jan ' : ' jangan ',
' janjin ' : ' janji ',
' jbjb ' : ' nimbrung ',
' jdi ' : ' jadi ',
' je ' : ' saja ',
' jeblok ' : ' jelek ',
' jelasin ' : ' jelas ',
' jengkel ' : ' kesal ',
' jenjang ' : ' tingkatan ',
' jg ' : ' juga ',
' jgak ' : ' juga ',
' jgn ' : ' jangan ',
' jidi ' : ' jadi ',
' jk ' : ' jika ',
' job ' : ' kerjaan ',
' join ' : ' gabung ',
' jugak ' : ' juga ',
' juge ' : ' juga ',
' just ' : ' hanya ',
' jwb ' : ' jawab ',
' jwban ' : ' jawaban ',
' kaga ' : ' tidak ',
' kaget ' : ' terkejut ',
' kak ' : ' kakak ',
' kaka ' : ' kakak ',
' kakaku ' : ' kakak saya ',
' kakel ' : ' kakak kelas ',
' kalem ' : ' diam ',
'kalo' : ' kalau ',
#' kan ' : ' ',
'kangen' : ' rindu ',
' karna ' : ' karena ',
' kartul ' : ' karya tulis ',
' kasi ' : ' kasih ',
' kasin ' : ' kasih ',
' kasitau ' : ' kasih tau ',
' kasta ' : ' tingkatan ',
' kat ' : ' kata ',
' kate ' : ' kata ',
' kau ' : ' kamu ',
' kaya ' : ' seperti ',
' kayak ' : ' seperti ',
' kdg ' : ' kadang ',
' kea ' : ' seperti ',
' keganggu ' : ' ganggu ',
' gangu ' : ' ganggu ',
' kehandle ' : ' teratur ',
' keinget ' : ' teringat ',
' kek ' : ' seperti ',
' bokek ' : ' krisis keuangan ',
' kelar ' : ' selesai ',
' kelarin ' : ' selesai ',
' kelelep ' : ' tenggelam ',
' kelon ' : ' kelas daring ',
' kelulus ' : ' lulus ',
' kemaren ' : ' kemarin ',
' kendala ' : ' masalah ',
' kepingin ' : ' ingin ',
' kerasa ' : ' terasa ',
' kerasin ' : ' keras ',
' kerjan ' : ' kerjaan ',
' kesanya ' : ' seperti ',
' ketemu ' : ' bertemu ',
' ketidakhadiran ' : ' bolos ',
' ketiwi ' : ' tawa ',
' kgn ' : ' rindu ',
' kiler ' : ' galak ',
' kils ' : ' membunuh ',
' matematik ' : ' matematika ',
' kl ' : ' kalau ',
' klw ' : ' kalau ',
'kls' : 'kelas',
'ksl' : 'kesal',
' km ' : ' kamu ',
' kmbali ' : ' kembali ',
' kmpul ' : ' kumpul ',
' kmrin ' : ' kemarin ',
' kmu ' : ' kamu ',
' knp ' : ' kenapa ',
' knpa ' : ' kenapa ',
' kode ' : ' sandi ',
' kompleks ' : ' rumit ',
' komplen ' : ' keluhan ',
' koneksi ' : ' hubungan ',
' koplak ' : ' lucu ',
' krn ' : ' karena ',
' ksh ' : ' kasih ',
' ku ' : ' saya ',
' kudu ' : ' harus ',
' kulon ' : ' kuliah daring ',
'kunyuk' : 'bodoh',
' ky ' : ' seperti ',
' kyak ' : ' seperti ',
' kyaknya ' : ' seperti ',
' laen ' : ' lain ',
' lahi ' : ' lagi ',
' lambat ' : ' lamban ',
' langanan ' : ' langganan ',
' lanjur ' : ' lanjut ',
' laprak ' : ' laporan praktik ',
' last ' : ' terakhir ',
' laper ' : ' lapar ',
' lbh ' : ' lebih ',
' lbih ' : ' lebih ',
' learners ' : ' pelajar ',
' learning ' : ' belajar ',
' least ' : ' sedikit ',
' leave ' : ' meninggalkan ',
' lecturer ' : ' dosen ',
' legal ' : ' resmi ',
' legends ' : ' sejarah ',
' lemes ' : ' lemas ',
' lemot ' : ' lambat ',
' les ' : ' kursus ',
' lg ' : ' lagi ',
'lgi' : ' lagi ',
' lgsg ' : ' langsung ',
' lgsng ' : ' langsung ',
' lgsung ' : ' langsung ',
' lgusng ' : ' langsung ',
#' lh ' : ' ',
#' la ' : ' ',
'liat' : 'lihat',
' light ' : ' cahaya ',
' like ' : ' suka ',
' line ' : ' garis ',
' list ' : ' daftar ',
' litle ' : ' kecil ',
' lngsng ' : ' langsung ',
' lo ' : ' kamu ',
' long ' : ' panjang ',
' loudspeaker ' : ' pengeras suara ',
' love ' : ' cinta ',
' low ' : ' lemah ',
' lu ' : ' kamu ',
' luama ' : ' lambat ',
' luck ' : ' beruntung ',
' lumayan ' : ' sedikit ',
' luringnya ' : ' luring ',
' lwt ' : ' lewat ',
' maap ' : ' maaf ',
' maba ' : ' mahasiswa baru ',
' mabar ' : ' main bersama ',
' mabok ' : ' penat ',
' maen ' : ' main ',
' maf ' : ' maaf ',
' mager ' : ' malas gerak ',
' mageran ' : ' malas gerak ',
' mahamin ' : ' memahami ',
' makas ' : ' terimakasih ',
' makasih ' : ' terimakasih ',
' maklumin ' : ' maklumi ',
' malem ' : ' malam ',
'males' : ' malas ',
' mama ' : ' ibu ',
' mamah ' : ' ibu ',
' mamak ' : ' ibu ',
' mandiri ' : ' mandiri ',
' manfatin ' : ' memanfaatkan ',
' manjur ' : ' ampuh ',
' mantepin ' : ' mantap ',
' manti ' : ' nanti ',
' mapel ' : ' mata pelajaran ',
' marem ' : ' kemarin ',
' masalahin ' : ' masalah ',
' masi ' : ' masih ',
' matdas ' : ' matematika dasar ',
' mateng ' : ' matang ',
' matika ' : ' matematika ',
' matkul ' : ' mata kuliah ',
' matpel ' : ' matapelsajaran ',
' maybe ' : ' mungkin ',
' mba ' : ' mbak ',
' mbok ' : ' mbak ',
' mcm ' : ' macam ',
' me ' : ' saya ',
' meet ' : ' bertemu ',
' megang ' : ' genggam ',
' mekap ' : ' riasan ',
' melajari ' : ' pelajari ',
' melukin ' : ' memeluk ',
' mempan ' : ' ampuh ',
' mencet ' : ' menekan ',
' menco ' : ' mencoba ',
' mending ' : ' sebaiknya ',
' mengangu ' : ' ganggu ',
' mentalitas ' : ' mental ',
' mentality ' : ' mental ',
' mepet ' : ' dekat ',
' merhatin ' : ' memerhatikan ',
' meriksa ' : ' periksa ',
' merta ' : ' serta ',
' mesti ' : ' harus ',
' musti ' : ' harus ',
' met ' : ' selamat ',
' meting ' : ' pertemuan ',
' mgkin ' : ' mungkin ',
' mikir ' : ' pikir ',
' mikirin ' : ' pikir ',
' milih ' : ' pilih ',
' mimum ' : ' minum ',
' min ' : ' admin ',
' mingdep ' : ' minggu depan ',
' mingu ' : ' minggu ',
' minjem ' : ' minjam ',
' ml ' : ' terus ',
' mulu ' : ' terus ',
' mlm ' : ' malam ',
' mnt ' : ' menit ',
' mo ' : ' mau ',
' molor ' : ' tidur ',
' more ' : ' lebih ',
' morning ' : ' pagi ',
' mosok ' : ' masa ',
' mpk ' : ' organisasi ',
' mreka ' : ' mereka ',
' mret ' : ' maret ',
' msh ' : ' masih ',
' msi ' : ' masih ',
' msie ' : ' masih ',
' msk ' : ' masak ',
' mslh ' : ' masalah ',
' mtk ' : ' matematika ',
' mu ' : ' kamu ',
' muak ' : ' sangat bosan ',
' mudeng ' : ' paham ',
' mudengin ' : ' paham ',
' muka ' : ' wajah ',
' mula ' : ' awal ',
' mumpung ' : ' kebetulan ',
' murids ' : ' murid ',
' mute ' : ' bisu ',
' muter ' : ' mutar ',
' my ' : ' saya ',
' naik ' : ' naik ',
' nak ' : ' anak ',
' nanya ' : ' tanya ',
' nder ' : ' pengirim ',
' nemenin ' : ' menemani ',
' nemu ' : ' temu ',
' nentuin ' : ' menentukan ',
' nerangin ' : ' menjelaskan ',
' nervous ' : ' gugup ',
' nga ' : ' tidak ',
' ngabarin ' : ' kabarin ',
' ngalamin ' : ' mengalami ',
' ngambis ' : ' ambis ',
' ngangkat ' : ' angkat ',
' nganter ' : ' antar ',
' nganterin ' : ' antar ',
' ngantuk ' : ' kantuk ',
' ngapain ' : ' sedang apa ',
' ngaret ' : ' terlambat ',
' ngatasinya ' : ' mengatasi ',
' ngebentak ' : ' membentak ',
' ngeblank ' : ' tidak fokus ',
' ngecewain ' : ' mengecewakan ',
' ngedown ' : ' putus asa ',
' ngejawab ' : ' menjawab ',
' ngejoke ' : ' menghibur ',
' ngelanjutin ' : ' lanjut ',
'ngerasa' : 'merasa',
' ngerasain ' : ' merasakan ',
' ngerekam ' : ' rekam ',
' ngerjain ' : ' mengerjakan ',
' ngerti ' : ' paham ',
' ngeselin ' : ' kesal ',
' ngeshare ' : ' bagikan ',
' ngga ' : ' tidak ',
' nghemat ' : ' hemat ',
' ngikut ' : ' ikut ',
' ngilangin ' : ' hilang ',
' ngingetin ' : ' mengingatkan ',
' ngirim ' : ' mengirim ',
' ngmong ' : ' bicara ',
' ngobrol ' : ' berbicara ',
' ngopi ' : ' kopi ',
' ngp ' : ' tidak apa apa ',
' ngrim ' : ' kirim ',
' ngs ' : ' nangis ',
' ngurus ' : ' urus ',
' ni ' : ' ini ',
' nie ' : ' ini ',
' nih ' : ' ini ',
' nilah ' : ' inilah ',
' ninbrung ' : ' ikut serta ',
' ninggalin ' : ' tinggalin ',
' ninjaku ' : ' pintasku ',
' nitip ' : ' titip ',
' nmor ' : ' nomor ',
' no ' : ' tidak ',
' nobar ' : ' lihat bersama ',
' nonton ' : ' lihat ',
' noh ' : ' itu ',
' nongki ' : ' kumpul ',
' nongkrong ' : ' tongkrong ',
' nope ' : ' tidak ',
' not ' : ' tidak ',
' notes ' : ' catatan ',
' nton ' : ' lihat ',
' ntr ' : ' nanti ',
' nugas ' : ' tugas ',
' nunggu ' : ' tunggu ',
' nungu ' : ' tunggu ',
' nunguin ' : ' tunggu ',
' nunjuk ' : ' tunjuk ',
#' ny ' : ' ',
' nyambung ' : ' sambung ',
' nyampe ' : ' sampai ',
' nyantai ' : ' santai ',
' nyantol ' : ' terikat ',
' nyari ' : ' cari ',
' nyasar ' : ' sasar ',
' nyatet ' : ' catat ',
' nyaut ' : ' saut ',
' nyeletuk ' : ' celetuk ',
' nyeremin ' : ' serem ',
' nyesel ' : ' sesal ',
' nyetir ' : ' setir ',
' nyetirnya ' : ' setir ',
' nyicil ' : ' cicil ',
' nyimak ' : ' simak ',
' nympe ' : ' sampai ',
' nyontek ' : ' contek ',
' nyuci ' : ' cuci ',
' nyuruh ' : ' suruh ',
' nyusahin ' : ' susahin ',
' nyusup ' : ' masuk ',
' oake ' : ' pakai ',
' obyek ' : ' objek ',
' ofcam ' : ' kamera mati ',
' offline ' : ' luring ',
'ofline' : 'luring',
' ogah ' : ' tidak mau ',
#' oiya ' : ' ',
' okay ' : ' siap ',
' okr ' : ' siap ',
' oke ' : ' siap ',
' olshopan ' : ' belanja ',
' on ' : ' daring ',
' oncam ' : ' kamera  hidup',
'oneline' : ' daring ',
' onlan ' : ' daring ',
'onlen' : ' daring ',
' onli ' : ' daring ',
'online' : ' daring ',
' open ' : ' buka ',
' opo ' : ' apa ',
' option ' : ' pilihan ',
' or ' : ' atau ',
' org ' : ' orang ',
' orng ' : ' orang ',
' ortu ' : ' orang tua ',
' ortuku ' : ' orangtua saya ',
' osis ' : ' organisasi ',
' our ' : ' kita ',
' over ' : ' berlebihan ',
' overthingking ' : ' banyak berpikir ',
' overthinking ' : ' banyak berpikir ',
' pa ' : ' bapak ',
' padal ' : ' padahal ',
' padet ' : ' padat ',
' pakat ' : ' paket ',
' pakat ' : ' sepakat ',
' pake ' : ' pakai ',
' palagi ' : ' apa lagi ',
' paling ' : ' sangat ',
' pandemic ' : ' pandemi ',
' panic ' : ' panik ',
' pantes ' : ' pantas ',
' paper ' : ' makalah ',
' parenting ' : ' mengasuh ',
' pas ' : ' ketika ',
' pas ' : ' saat ',
' patu ' : ' patuh ',
' pd ' : ' pada ',
' pda ' : ' pada ',
' pdhal ' : ' padahal ',
' pdhl ' : ' padahal ',
' peace ' : ' damai ',
' pearson ' : ' orang ',
' pede ' : ' berani ',
' pedenya ' : ' percaya diri ',
' pedes ' : ' pedas ',
' pedulin ' : ' peduli ',
' pelajaranya ' : ' ajar ',
' pemblajaran ' : ' pembelajaran ',
' pemperov ' : ' pemerintah provinsi ',
' penangung ' : ' tanggung ',
' pencet ' : ' tekan ',
'pengen' : 'ingin',
'pengin' : 'ingin',
' pening ' : ' penat ',
' person ' : ' orang ',
' personilnya ' : ' anggotanya ',
' pewe ' : ' kata sandi ',
' pg ' : ' pilihan ganda ',
' pgn ' : ' ingin ',
' phonenya ' : ' telpon ',
' pingang ' : ' pinggang ',
' pinter ' : ' pintar ',
' pinternya ' : ' pintar ',
' cerdas ' : ' pintar ',
' nulis ' : ' tulis ',
' piye ' : ' bagaimana ',
' plg ' : ' pulang ',
' plis ' : ' mohon ',
' pljrn ' : ' pelajaran ',
' pls ' : ' mohon ',
' plus ' : ' dan ',
' pny ' : ' punya ',
' pnya ' : ' milik ',
' porto ' : ' portofolio ',
' portopolio ' : ' portofolio ',
' poster ' : ' gambar ',
' postes ' : ' latihan ',
' potensi ' : ' potensi ',
' praktek ' : ' praktik ',
' prefer ' : ' pilih ',
' prepare ' : ' mempersiapkan ',
' present ' : ' kehadiran ',
' pretes ' : ' latihan ',
' prioritas ' : ' utama ',
' progres ' : ' proses ',
' ps ' : ' saat ',
' pungung ' : ' punggung ',
' puyeng ' : ' penat ',
' qaqa ' : ' kakak ',
' question ' : ' pertanyaan ',
' quiet ' : ' cukup ',
' rada ' : ' sedikit ',
' rame ' : ' ramai ',
' rank ' : ' peringkat ',
' rapi ' : ' rapih ',
' realife ' : ' dunia nyata ',
' record ' : ' rekaman ',
' refreshing ' : ' santai ',
' regret ' : ' sesal ',
' rejekinya ' : ' rezeki ',
' rekoemneasi ' : ' rekomendasi ',
' rekomen ' : ' rekomendasi ',
' remed ' : ' remedial ',
'respon' : ' balas ',
' rest ' : ' istirahat ',
#' ri ' : '',
' rileks ' : ' santai ',
' risalah ' : ' risalah ',
' riweh ' : ' ribet ',
' riweuh ' : ' ribet ',
' ribetnya ' : ' ribet ',
' diribetin ' : ' ribet ',
' rl ' : ' dunia nyata ',
' rmh ' : ' rumah ',
' rupe ' : ' rupa ',
' sabi ' : ' bisa ',
' sad ' : ' nangis ',
' salty ' : ' tidak sopan ',
' sambi ' : ' sambil ',
' sampe ' : ' sampai ',
' sampeguru ' : ' sampai guru ',
' sampek ' : ' sampai ',
' samsek ' : ' samasekali ',
' sangup ' : ' sanggup ',
' sapatau ' : ' siapa tahu ',
' sat ' : ' saat ',
' saur ' : ' sahur ',
' sbenarnya ' : ' sebenarnya ',
' sbg ' : ' sebagai ',
' sblum ' : ' sebelum ',
' scangi ' : ' canggih ',
' sch ' : ' sekolah ',
' schol ' : ' sekolah ',
' scr ' : ' secara ',
' sd ' : ' sekolah dasar ',
' sdn ' : ' sedan ',
' sefrekuensi ' : ' sejenis ',
' seger ' : ' segar ',
' fresh ' : ' segar ',
' sekola ' : ' sekolah ',
' sekre ' : ' sekretaris ',
' sekul ' : ' sekolah ',
' selese ' : ' selesai ',
' semingu ' : ' seminggu ',
' sempet ' : ' sempat ',
' sempetin ' : ' sempat ',
' sempetnya ' : ' sempat ',
' semseter ' : ' semester ',
' semster ' : ' semester ',
' sender ' : ' pengirim ',
' sendiriwkwkwkwk ' : ' sendiri ',
' seneng ' : ' senang ',
' sengaknya ' : ' setidaknya ',
' senyap ' : ' bisu ',
' sepreti ' : ' seperti ',
' setidanya ' : ' setidaknya ',
' sharing ' : ' berbagi ',
' shock ' : ' terkejut ',

' sider ' : ' pembaca ',
' silibgram ' : ' selebgram ',
' simulasi ' : ' latihan ',
' since ' : ' sejak ',
' since ' : ' selama ',
' sing ' : ' nyayi ',
' sirkel ' : ' lingkungan ',
' sis ' : ' kakak ',
' skali ' : ' sekali ',
' sklh ' : ' sekolah ',
' skolah ' : ' sekolah ',
' skrg ' : ' sekarang ',
' skul ' : ' sekolah ',
' sm ' : ' sama ',
' sma ' : ' sekolah menengah akhir ',
' smgt ' : ' semangat ',
' smk ' : ' sekolah menengah kejuruan ',
' smngat ' : ' semangat ',
' smp ' : ' sekolah menengah pertama ',
' smpe ' : ' sampai ',
' smpek ' : ' sampai ',
' smt ' : ' semester ',
' smua ' : ' semua ',
' solution ' : ' solusi ',
' sosmed ' : ' sosial media ',
' speak ' : ' berbicara ',
' speaker ' : ' pengeras suara ',
' sprti ' : ' seperti ',
' spy ' : ' supaya ',
' stay ' : ' tingal ',
' stil ' : ' masih ',
' stlh ' : ' setelah ',
' stop ' : ' berhenti ',
' story ' : ' cerita ',
'stres' : 'penat',
' tres ' : ' penat ',
'pusing' : 'penat',
' struglenya ' : ' perjuanganya ',
' stud ' : ' belajar ',
' student ' : ' murid ',
' study ' : ' belajar ',
' style ' : ' model ',
' sunguh ' : ' sungguh ',
' susahla ' : ' susah ',
' syok ' : ' terkejut ',
' syp ' : ' siapa ',
' tafaham ' : ' tidak paham ',
' taip ' : ' tapi ',
' tak ' : ' tidak ',
' take ' : ' mengambil ',
' tang ' : ' yang ',
' tanga ' : ' tangga ',
' tanganya ' : ' tangan ',
' tangung ' : ' tanggung ',
' tapu ' : ' tapi ',
' tar ' : ' nanti ',
' tau ' : ' tahu ',
' taun ' : ' tahun ',
' tb ' : ' tiba ',
' tba ' : ' tiba ',
' tbh ' : ' tubuh ',
' tbtb ' : ' tibatiba ',
' tdi ' : ' tadi ',
' tdk ' : ' tidak ',
' tdr ' : ' tidur ',
' rebahan ' : ' tidur ',
' teacher ' : ' guru ',
' teachers ' : ' guru ',
' tekhniknya ' : ' tehniknya ',
' telepon ' : ' telpon ',
' terkhir ' : ' terakhir ',
' teros ' : ' lalu ',
' tertingal ' : ' tertinggal ',
' tes ' : ' uji ',
'tetanga' : 'tetangga',
' tetep ' : ' tetap ',
' tetiba ' : ' tibatiba ',
' teu ' : ' tidak ',
' tgl ' : ' tanggal ',
' th ' : ' tahun ',
' thank ' : ' terimakasih ',
#' the ' : ' ',
' thead ' : ' utas ',
' then ' : ' lalu ',
' there ' : ' sana ',
' thinking ' : ' berfikir ',
' third ' : ' ketiga ',
' this ' : ' ini ',
' thread ' : ' utas ',
' throw ' : ' lempar ',
' tiap ' : ' setiap ',
' tingal ' : ' tinggal ',
' tingi ' : ' tinggi ',
' tipe ' : ' jenis ',
' tips ' : ' cara ',
' tkt ' : ' takut ',
' tl ' : ' beranda ',
' tmbh ' : ' tumbuh ',
' tmen ' : ' teman ',
' tmn ' : ' teman ',
' toxic ' : ' racun ',
' tp ' : ' tapi ',
' tpi ' : ' tapi ',
' tps ' : ' tipis ',
' tpt ' : ' tepat ',
' tptnya ' : ' tepat ',
' treatment ' : ' melatih ',
' tremor ' : ' gugup ',
'deg degan' : 'gugup',
' tres ' : ' penat ',
' trick ' : ' cara ',
' trs ' : ' lalu ',
' trspas ' : ' terus waktu ',
' trus ' : ' lalu ',
' try ' : ' coba ',
' tu ' : ' itu ',
' tube ' : ' tabung ',
' tuh ' : ' itu ',
' tulong ' : ' tolong ',
' turns out ' : ' ternyata ',
' twet ' : ' tweet ',
' twt ' : ' tweet ',
' uas ' : ' ujian akhir sekolah ',
' ud ' : ' sudah ',
' udah ' : ' sudah ',
' udahmah ' : ' sudah ',
' udeh ' : ' sudah ',
'udh' : ' sudah ',
' ugh ' : ' ',
' ugha ' : ' juga ',
' uh ' : ' ujian harian ',
' ukm ' : ' unit kegiatan mahasiswa ',
' ultah ' : ' ulang tahun ',
'diaorg' : ' dia orang ',
' un ' : ' uji nasional ',
' understand ' : ' paham ',
' unexpected ' : ' tidak disangka ',
' ungulan ' : ' unggulan ',
' unk ' : ' untuk ',
' unmod ' : ' kesal',
' unmute ' : ' bersuara ',
' unruk ' : ' untuk ',
' untk ' : ' untuk ',
' unyu ' : ' lucu ',
' update ' : ' terkini ',
' upload ' : ' unggah ',
' upluged ' : ' dicabut ',
' us ' : ' kita ',
' ut ' : ' untuk ',
' utbk ' : ' uji tulis berbasis komputer ',
' utk ' : ' untuk ',
' uwueheueu ' : ' ',
' validasi ' : ' kebenaran ',
' via ' : ' melalui ',
' ngakak ' : ' tawa ',
' volume ' : ' suara ',
' wa ' : ' whatsapps ',
' wae ' : ' ',
' wah ' : ' kagum ',
' waiting ' : ' menunggu ',
' walas ' : ' wali kelas ',
' walopun ' : ' walaupun ',
' walu ' : ' walau ',
' waro ' : ' pandang ',
' was ' : ' ',
' water ' : ' air ',
' wekend ' : ' hari libur ',
' welcome ' : ' selamat datang ',
' what ' : ' apa ',
' whatsap ' : ' whatsapps ',
' why ' : ' kenapa ',
' wifi ' : ' internet ',
' wih ' : ' kagum ',
' win ' : ' menang ',
' wishmeluck ' : ' harap saya berhasil ',
' wisudan ' : ' wisuda ',
' with ' : ' dengan ',
' wkt ' : ' waktu ',
' wktu ' : ' waktu ',
 'wkwk ' : ' ',
' wlpn ' : ' walaupun ',
' woi ' : ' ',
' woy ' : ' ',
' wtf ' : ' bangsat ',
' wuhu ' : ' bersorak ',
' xixixi ' : ' ',
' iya ' : ' ya ',
' yah ' : ' ',
' yahud ' : ' mantap ',
' yak ' : ' ya ',
' yakali ' : ' masa ya ',
' yalah ' : ' allah ',
' yamasa ' : ' masa ',
' yap ' : ' ya ',
' yauda ' : ' yasudah ',
' yaudah ' : ' sudah ',
' yawlah ' : ' allah ',
' yay ' : ' bersorak ',
' ye ' : ' bersorak ',
' yea ' : ' bersorak ',
' year ' : ' tahun ',
' yeay ' : ' hore ',
' yekan ' : ' yakan ',
' yes ' : ' ya ',
' yodeh ' : ' sudah ',
' yoi ' : ' ya ',
' you ' : ' kamu ',
' young ' : ' muda ',
' your ' : ' milik kamu ',
' yutupanpun ' : ' youtube ',
' zona ' : ' area ',
'aelah' : ' ',
'afsgaga' : ' ',
'afsgagahaha' : ' ',
'ajahshaja' : ' ',
'ajg' : ' anjing ',
'anje' : ' anjing ',
'anjeng' : ' anjing ',
'anjim' : ' anjing ',
'anjr' : ' anjing ',
'anying' : ' anjing ',
'asalamualaikum ' : ' salam ',
'asiq' : ' asik ',
'askfaskfajcaj' : ' ',
'askrl' : ' ',
'asyu' : ' anjing ',
'awkward' : ' canggung ',
'bagong' : ' besar ',
'bapa ' : ' bapak ',
'bgt' : ' sangat ',
'bljr' : ' belajar ',
'doang' : ' ',
'eh ' : ' ',
'gada' : ' tidak ada ',
'hah' : ' ',
'haha' : '',
'hahaha' : '',
'hahahah' : '',
'hahahaha' : ' ',
'hahahahah' : ' ',
'hahahahahahahaha' : '',
'hehe' : ' ',
' klu ' : ' kalau ',
'jd ' : ' jadi ',
' kawan ' : ' teman ',
'kenang' : ' kenang ',
'klo' : ' kalau ',
'kn ' : 'kan ',
' met ' : ' selamat ',
'papah' : ' bapak ',
'private' : 'pribadi',
'privat' : 'pribadi',
'riweh' : ' ribet ',
' temen ' : ' teman ',
'ujikom' : ' uji kompetensi ',
' ukom ' : ' uji kompetensi ',
'ulangam' : ' uji ',
'ulngan' : ' uji ',
'uprak' : ' uji praktik ',
#'wk' : ' ',
'wkwk' : ' ',
'wkwkw' : ' ',
'wkwkwk' : ' ',
'yampun' : ' astaga ',
'yg' : ' yang ',
' yng ' : ' yang ',
' yt ' : ' youtube ',
' yutub ' : ' youtube ',
'zom' : ' zoom ',
'sekedar' : 'hanya',
#' mah ' : ' ',
'bentar' : 'singkat',
' huhu ' : ' nangis ',
' alhamdulilah ' : ' syukur ',
' selesa ' : ' selesai ',
' gmet ' : ' googlemeet ',
'rajin' : 'giat',
'ngomong' : 'bicara',
' unmood ' : ' kesal ',
' mod ' : ' perasaan ',
' mody ' : ' perasaan ',
' ngga mood ' : ' malas ',
' badmood ' : ' kesal ',
' badmod ' : ' kesal ',
' gt ' : ' begitu ',
' bilang ' : ' bicara ',
' asa ' : ' ada ',
' bingungin ' : ' bingung ',
' asik ' : ' senang ',
' kalem ' : ' santai ',
' ngeluh ' : ' keluh ',
' ketingalan ' : ' ketinggalan ',
' time ' : ' waktu ',
'janlup' : ' jangan lupa ',
' tungu ' : ' tunggu ',
' ngajar ' : ' ajar ',
' ngajarin ' : ' ajar ',
' kesel ' : ' kesal ',
'pro' : ' dukung ',
'kontra' : ' lawan ',
' ngasih ' : ' kasih ',
' ah ' : ' ',
' asyu ' : ' anjing ',
' pewe ' : ' nyaman ',
'betah' : 'nyaman',
' panic ' : ' panik ',
' semngat ' : ' semangat ',
' lamban ' : ' lambat ',
' mantap ' : ' bagus ',
' terima kasih ' : ' terimakasih ',
' lect ' : ' dosen ',
' semangaat ' : ' semangat ',
' semangatt ' : ' semangat ',
' ndak ' : ' tidak ',
' sarapan ' : ' makan ',
' astagfirullah ' : ' astaga ',
' apeni ' : ' apa ',
' dak ' : ' ',

}

In [ ]:
#Case Folding
df['Tweet'] = df['Tweet'].apply(lambda i : i.lower())

#Data Cleaning
def emoticon(tweet,emot):
    for i,j in emot.items():
        tweet = tweet.replace(i,j)
    return tweet
df['Tweet'] = [emoticon(x,daftar_emot) for x in df['Tweet']]

def datacleaning_word(tweet):
  #link
  link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
  links         = re.findall(link_regex, tweet)
    
  for link in links:
     tweet = tweet.replace(link[0], ' ')
  
  #username
  tweet = re.sub(r'@[\S]+', '', tweet)
  #hastag
  tweet = re.sub(r'#[\S]+', '', tweet)
   # Replace multiple spaces with a single space
  tweet = re.sub(r'\s+', ' ', tweet)
  # Convert more than 2 letter repetitions to 2 letter
    # funnnnny --> funny
  tweet = re.sub(r'(.)\1+', r'\1', tweet)
  tweet = re.sub(r"[^a-zA-Z]+", " ", tweet)
  tweet = tweet.replace('\n', ' ')

  return tweet
df['Tweet'] = [datacleaning_word(x) for x in df['Tweet']]

#normalization
def normalization(tweet,kamus):
    for i,j in kamus.items():
        tweet = tweet.replace(i,j)
    return tweet
df['Tweet'] = [normalization(x,kamus_kata) for x in df['Tweet']]

# stopword removal
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()
df['Tweet'] = [stopword.remove(s) for s in df['Tweet']]

#stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()
df['Tweet'] = [stemmer.stem(s) for s in df['Tweet']]

#Tokenizing
df['TweetToken'] = [word_tokenize(x) for x in df['Tweet']]

In [ ]:
df

,Tweet,Label,TweetToken
0,kelas daring kini kagum,1,"[kelas, daring, kini, kagum]"
1,sistem ajar daring daring anak pasuk kelas sek...,0,"[sistem, ajar, daring, daring, anak, pasuk, ke..."
2,daring scrol tiktok suara menang biasa nyanyin...,0,"[daring, scrol, tiktok, suara, menang, biasa, ..."
3,ara lupa makan lemas daring kelas hm pokok sem...,1,"[ara, lupa, makan, lemas, daring, kelas, hm, p..."
4,kemarin kelas daring nendang botcamp bingung k...,1,"[kemarin, kelas, daring, nendang, botcamp, bin..."
...,...,...,...
695,kelas daring semangat,1,"[kelas, daring, semangat]"
696,yuk cepat kelas daring yuk pikirin otak pikir ...,1,"[yuk, cepat, kelas, daring, yuk, pikirin, otak..."
697,ape semenjak kelas daring sakit kepala sedih d...,0,"[ape, semenjak, kelas, daring, sakit, kepala, ..."
698,hari kamar kelas daring santai santai kamar ayu,1,"[hari, kamar, kelas, daring, santai, santai, k..."


#**Pembobotan TF-IDF**

In [ ]:
# calc TF vector
cvect = CountVectorizer()
TF_vector = cvect.fit_transform(df['Tweet'])

# normalize TF vector
normalized_TF_vector = normalize(TF_vector, norm='l1', axis=1)

# calc IDF
tfidf = TfidfVectorizer(max_df = 1.0, min_df= 1 ,norm= None,smooth_idf=True,use_idf=True)
tfs = tfidf.fit_transform(df['Tweet'])
IDF_vector = tfidf.idf_
feature_names = tfidf.get_feature_names()

# hitung TF x IDF sehingga dihasilkan TFIDF matrix / vector
tfidf_mat = normalized_TF_vector.multiply(IDF_vector).toarray()

tabel_tf = pd.DataFrame(np.transpose(normalized_TF_vector.A),index= feature_names)
tabel_idf = pd.DataFrame(np.transpose(IDF_vector),index= feature_names,columns=['IDF'])
tabel_tfidf = pd.DataFrame(np.transpose(tfidf_mat),index= feature_names)

tabel_gabungan = pd.concat([tabel_tf,tabel_idf,tabel_tfidf], axis=1)


In [ ]:
  data = pd.DataFrame(np.transpose(tabel_tfidf))
  data['label'] = df['Label']

# **Pembagian dan Pemodelan Data**

In [ ]:
fitur = np.array(feature_names)
headers = list(data.columns.values)
X = data[headers[:-1]]
y = data[headers[-1:]].values.ravel()

In [ ]:
from sklearn.model_selection import StratifiedKFold
# set to 10 folds
skf = StratifiedKFold(n_splits=10)

# blank lists to store predicted values and actual values
predicted_y = []
expected_y = []

i=1
# partition data
for train_index, test_index in skf.split(X, y):
    print("ITERASI KE-",i)
    # specific ".loc" syntax for working with dataframes
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print('Training dim:',X_train.shape , 'Test dim:', X_test.shape)
    
 
    # create and fit classifier
    model =SVC(C=1, break_ties=False, cache_size=200, class_weight=None,
               coef0=0.0, decision_function_shape='ovr', degree=3, gamma=0.1,
               kernel='linear', max_iter=-1, probability=False, random_state=None,
               shrinking=True, tol=0.001, verbose=False)
    model.fit(X_train, y_train)

    prediksi = model.predict(X_test)

    # store expected result for this specific fold
    expected_y.extend(y_test)
    
    print(metrics.classification_report(y_test, prediksi))
    print(metrics.confusion_matrix(y_test, prediksi))
      
    # store result from classification
    predicted_y.extend(prediksi)
    i = i + 1


ITERASI KE- 1
Training dim: (630, 1881) Test dim: (70, 1881)
              precision    recall  f1-score   support

           0       0.81      0.86      0.83        35
           1       0.85      0.80      0.82        35

    accuracy                           0.83        70
   macro avg       0.83      0.83      0.83        70
weighted avg       0.83      0.83      0.83        70

[[30  5]
 [ 7 28]]
ITERASI KE- 2
Training dim: (630, 1881) Test dim: (70, 1881)
              precision    recall  f1-score   support

           0       0.85      0.63      0.72        35
           1       0.70      0.89      0.78        35

    accuracy                           0.76        70
   macro avg       0.78      0.76      0.75        70
weighted avg       0.78      0.76      0.75        70

[[22 13]
 [ 4 31]]
ITERASI KE- 3
Training dim: (630, 1881) Test dim: (70, 1881)
              precision    recall  f1-score   support

           0       0.75      0.77      0.76        35
           1    